## What effect does a Airbnb host’s bio metadata have on it’s frequency of their rentals being booked?


In [1]:
import csv
import pandas as pd
import numpy as np


In [11]:
# import data
calendar_all = pd.read_csv('./data/calendar.csv')
listings_all = pd.read_csv('./data/listings.csv')

# reviews = pd.read_csv('./data/reviews.csv')


In [12]:
# do a bit of cleaning

# remove cols that only contain (1) unique elem
listings = listings_all
for col in listings_all.columns:
    if len(listings_all[col].unique()) == 1:
        listings = listings.drop(col,axis=1)

In [17]:
listings.head()

# list(listings.columns.values)

# listings['host_location'].unique()



,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,thumbnail_url,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,NaN,NaN,NaN,NaN,...,10.0,10.0,9.0,10.0,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,...,10.0,10.0,10.0,10.0,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,NaN,...,10.0,10.0,10.0,10.0,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,We are in the beautiful neighborhood of Queen ...,Belltown,The nearest public transit bus (D Line) is 2 b...,NaN,...,10.0,10.0,9.0,9.0,f,strict,f,f,1,0.89


In [18]:
host_data = listings[['host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified']]

host_data.head()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t
1,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t
2,16708587,https://www.airbnb.com/users/show/16708587,Jill,2014-06-12,"Seattle, Washington, United States",i love living in Seattle. i grew up in the mi...,within a few hours,67%,100%,f,https://a1.muscache.com/ac/users/16708587/prof...,https://a1.muscache.com/ac/users/16708587/prof...,Queen Anne,2.0,2.0,"['email', 'phone', 'google', 'reviews', 'jumio']",t,t
3,9851441,https://www.airbnb.com/users/show/9851441,Emily,2013-11-06,"Seattle, Washington, United States",NaN,NaN,NaN,NaN,f,https://a2.muscache.com/ac/users/9851441/profi...,https://a2.muscache.com/ac/users/9851441/profi...,Queen Anne,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t
4,1452570,https://www.airbnb.com/users/show/1452570,Emily,2011-11-29,"Seattle, Washington, United States","Hi, I live in Seattle, Washington but I'm orig...",within an hour,100%,NaN,f,https://a0.muscache.com/ac/users/1452570/profi...,https://a0.muscache.com/ac/users/1452570/profi...,Queen Anne,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t


In [29]:
# number of about sections that we have data for: 3817 - 858 = 2959
host_data['host_about'].count()

# only 7 hosts do not have a profile picture
host_data[host_data.host_has_profile_pic == 'f']

host_data['host_about']



0       I am an artist, interior designer, and run a s...
1       Living east coast/left coast/overseas.  Time i...
2       i love living in Seattle.  i grew up in the mi...
3                                                     NaN
4       Hi, I live in Seattle, Washington but I'm orig...
5       I've been fortunate to have had many life expe...
6       We have lived in the the Queen Anne neighborho...
7       I am a carpenter at heart , I love music , I p...
8       I'm a 35 year old professional from Seattle, W...
9       Living east coast/left coast/overseas.  Time i...
10      Eileen and Ray were born in Seattle just three...
11      My name is Nadia, and I am a native of Seattle...
12      We both grew up on Queen Anne Hill, then later...
13      Hi, my name is Ellie.  I’m a widow.  My husban...
14      Originally from Southern California, my husban...
15      I'm from Seattle.  I am a designer, inventor a...
16      I'm a 37 year old recent law school graduate, ...
17            